In [1]:
import os

import numpy as np
import pandas as pd
import zipfile
import tensorflow as tf
import shutil
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = os.path.abspath("./drive/MyDrive/MegaClassifier")

Mounted at /content/drive


In [3]:
MODEL_NAME = "MegaClassifier_b"
VERSION = "v1"

In [4]:
DATASETS = {
    "MegaClassifier_a": "onlyDetectionsForTrain",
    "MegaClassifier_b": "emptyOriginalAnimalDetection",
    "MegaClassifier_c": "emptyNonEmptyDataset",
}

In [5]:
DATASET_PATH_ZIP = os.path.join(BASE_PATH, f"data/processed/{DATASETS[MODEL_NAME]}.zip")
with zipfile.ZipFile(DATASET_PATH_ZIP, 'r') as zip_ref:
      zip_ref.extractall("./data/processed")
DATASET_DIR = os.path.abspath(f"data/processed/{DATASETS[MODEL_NAME]}")

In [6]:
EPOCHS = 10

IMAGE_SIZE = (456, 456)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [7]:
for BATCH_INDEX in range(4):

  OPTIMIZER = tf.keras.optimizers.Adam()

  LOSS_FUNCTION = tf.keras.losses.BinaryCrossentropy()

  METRICS = [
      BinaryAccuracy(name="accuracy"),
      Precision(name="precision"),
      Recall(name="recall"),
      AUC(name="auc"),
  ]

  BATCH_SIZES = [16, 32, 64, 128]
  BATCH_SIZE = BATCH_SIZES[BATCH_INDEX]

  SUBVERSION = VERSION + f".{BATCH_INDEX}"

  LOGS_PATH = os.path.abspath(f"./logs")
  if os.path.exists(LOGS_PATH):
    !rm -rf {LOGS_PATH}
  os.makedirs(LOGS_PATH, exist_ok=True)

  MODELS_PATH = os.path.abspath("./models")
  if os.path.exists(MODELS_PATH):
    !rm -rf {MODELS_PATH}
  os.makedirs(MODELS_PATH, exist_ok=True)


  train_datagen = ImageDataGenerator(
      preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
  )
  train_images = train_datagen.flow_from_directory(
    directory=f"{DATASET_DIR}/train",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    classes=['vacia', 'animal'],
    shuffle=True,
    seed=SEED,
  )

  datagen = ImageDataGenerator(
      preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
  )
  validation_images = datagen.flow_from_directory(
    directory=f"{DATASET_DIR}/validation",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    classes=['vacia', 'animal'],
    shuffle=False,
    seed=SEED,
  )

  pretrained_model = tf.keras.applications.EfficientNetB5(
      weights="imagenet",
      include_top=False,
      input_shape=IMAGE_SHAPE,
  )
  pretrained_model.trainable = False

  model = tf.keras.Sequential(
    [
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ],
    name=f"{MODEL_NAME}_{SUBVERSION}",
  )

  model.compile(
      optimizer=OPTIMIZER,
      loss=LOSS_FUNCTION,
      metrics=METRICS,
  )

  history = model.fit(
      train_images,
      epochs=EPOCHS,
      batch_size=BATCH_SIZE,
      validation_data=validation_images,
  )

  dataframe = pd.DataFrame(history.history)
  history_path = os.path.join(LOGS_PATH, f"history_{SUBVERSION}.csv")
  dataframe.to_csv(history_path, sep=";", index=False)


  model.save(os.path.join(MODELS_PATH,f"{MODEL_NAME}_{SUBVERSION}.weights.h5"))
  model.save(os.path.join(MODELS_PATH,f"{MODEL_NAME}_{SUBVERSION}.keras"))

  shutil.copytree(LOGS_PATH, os.path.join(BASE_PATH,f"logs/{MODEL_NAME}/{VERSION}/{SUBVERSION}"),dirs_exist_ok=True)
  shutil.copytree(MODELS_PATH, os.path.join(BASE_PATH,f"models/{MODEL_NAME}/{VERSION}/{SUBVERSION}"),dirs_exist_ok=True)

Found 19995 images belonging to 2 classes.
Found 4286 images belonging to 2 classes.
115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 283s 169ms/step - accuracy: 0.8874 - auc: 0.9503 - loss: 0.2719 - precision: 0.9191 - recall: 0.9100 - val_accuracy: 0.9414 - val_auc: 0.9845 - val_loss: 0.1546 - val_precision: 0.9675 - val_recall: 0.9432
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 121s 97ms/step - accuracy: 0.9444 - auc: 0.9863 - loss: 0.1461 - precision: 0.9640 - recall: 0.9509 - val_accuracy: 0.9452 - val_auc: 0.9864 - val_loss: 0.1408 - val_precision: 0.9690 - val_recall: 0.9475
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 126s 101ms/step - accuracy: 0.9478 - auc: 0.9888 - loss: 0.1308 - precision: 0.9669 - recall: 0.9539 - val_accuracy: 0.9477 - val_auc: 0.9870 - val_loss: 0.1359 - val_precision: 0.9628 - val_recall: 0.9580
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 120s 96ms/step - accuracy: 0.9467 - auc: 0.9893 - loss: 0.1273 - precision: 0.9646 - recall: 0.9542 - val_accuracy: 0.9487 - val_auc: 0.9878 - val_loss: 0.1316 - val_precision: 0.9648 - val_recall: 0.9573
Epoch 5/10

Found 19995 images belonging to 2 classes.
Found 4286 images belonging to 2 classes.
Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 297s 351ms/step - accuracy: 0.8731 - auc: 0.9378 - loss: 0.3041 - precision: 0.9043 - recall: 0.9073 - val_accuracy: 0.9393 - val_auc: 0.9824 - val_loss: 0.1668 - val_precision: 0.9663 - val_recall: 0.9411
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 123s 197ms/step - accuracy: 0.9367 - auc: 0.9837 - loss: 0.1608 - precision: 0.9584 - recall: 0.9453 - val_accuracy: 0.9421 - val_auc: 0.9856 - val_loss: 0.1474 - val_precision: 0.9641 - val_recall: 0.9478
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 123s 196ms/step - accuracy: 0.9469 - auc: 0.9879 - loss: 0.1374 - precision: 0.9658 - recall: 0.9541 - val_accuracy: 0.9475 - val_auc: 0.9864 - val_loss: 0.1409 - val_precision: 0.9641 - val_recall: 0.9563
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 124s 198ms/step - accuracy: 0.9510 - auc: 0.9894 - loss: 0.1277 - precision: 0.9673 - recall: 0.9588 - val_accuracy: 0.9487 - val_auc: 0.

Found 19995 images belonging to 2 classes.
Found 4286 images belonging to 2 classes.
Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 294s 643ms/step - accuracy: 0.8597 - auc: 0.9289 - loss: 0.3432 - precision: 0.9026 - recall: 0.8816 - val_accuracy: 0.9305 - val_auc: 0.9799 - val_loss: 0.1835 - val_precision: 0.9468 - val_recall: 0.9482
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 121s 385ms/step - accuracy: 0.9408 - auc: 0.9837 - loss: 0.1664 - precision: 0.9592 - recall: 0.9509 - val_accuracy: 0.9396 - val_auc: 0.9842 - val_loss: 0.1579 - val_precision: 0.9581 - val_recall: 0.9503
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 121s 387ms/step - accuracy: 0.9425 - auc: 0.9856 - loss: 0.1518 - precision: 0.9628 - recall: 0.9501 - val_accuracy: 0.9442 - val_auc: 0.9857 - val_loss: 0.1468 - val_precision: 0.9636 - val_recall: 0.9517
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 122s 388ms/step - accuracy: 0.9470 - auc: 0.9878 - loss: 0.1379 - precision: 0.9658 - recall: 0.9538 - val_accuracy: 0.9459 - val_auc: 0.

Found 19995 images belonging to 2 classes.
Found 4286 images belonging to 2 classes.
Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 276s 1s/step - accuracy: 0.8456 - auc: 0.9174 - loss: 0.3839 - precision: 0.8878 - recall: 0.8779 - val_accuracy: 0.9279 - val_auc: 0.9768 - val_loss: 0.2069 - val_precision: 0.9580 - val_recall: 0.9319
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 119s 758ms/step - accuracy: 0.9275 - auc: 0.9792 - loss: 0.1925 - precision: 0.9565 - recall: 0.9336 - val_accuracy: 0.9375 - val_auc: 0.9820 - val_loss: 0.1741 - val_precision: 0.9676 - val_recall: 0.9369
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 120s 763ms/step - accuracy: 0.9376 - auc: 0.9835 - loss: 0.1661 - precision: 0.9604 - recall: 0.9447 - val_accuracy: 0.9396 - val_auc: 0.9840 - val_loss: 0.1593 - val_precision: 0.9587 - val_recall: 0.9496
Epoch 4/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 120s 762ms/step - accuracy: 0.9399 - auc: 0.9854 - loss: 0.1538 - precision: 0.9599 - recall: 0.9490 - val_accuracy: 0.9431 - val_auc: 0.985